# I. Libraries

In [27]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import csv
import math
from math import log
import time
from tqdm import tqdm

# II. Load data

In [28]:
start_time = time.time()
np.random.seed(42)

In [ ]:
print("Loading node features...")
node_features = {}
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        # ID, 932 features
        node_id = int(float(row[0]))
        features = np.array([float(x) for x in row[1:]])  # 932 features
        node_features[node_id] = features

print(f"Loaded features for {len(node_features)} nodes")
print(f"Feature vector length: {len(next(iter(node_features.values())))}")

G = nx.Graph()

for node_id in node_features:
    G.add_node(node_id)

print("Building graph from training data...")
edges_positive = []
edges_negative = []
with open("train.txt", "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 3:  # Ensure proper format
            src, dst, label = map(int, parts)
            if label == 1:
                G.add_edge(src, dst)
                edges_positive.append((src, dst))
            else:
                edges_negative.append((src, dst))

print(f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
print(f"Positive edges: {len(edges_positive)}, Negative edges: {len(edges_negative)}")

print("Loading test data...")
test_pairs = []
with open("test.txt", "r") as f:
    reader = csv.reader(f)
    test_set = list(reader)
test_pairs = [tuple(map(int, element[0].split())) for element in test_set]
print(f"Loaded {len(test_pairs)} test pairs")



Loading node features...
Loaded features for 3597 nodes
Feature vector length: 932
Building graph from training data...
Graph built with 3597 nodes and 5248 edges
Positive edges: 5248, Negative edges: 5248
Loading test data...
Loaded 3498 test pairs


# II. Feature extraction function + custom metrics

In [30]:
def custom_adamic_adar_index(G, node1, node2):
    """Calculate Adamic-Adar index with proper error handling"""
    common_neighbors = list(nx.common_neighbors(G, node1, node2))
    if not common_neighbors:
        return 0
    
    score = 0
    for w in common_neighbors:
        degree = G.degree(w)
        # Avoid division by zero when log(1) = 0
        if degree > 1:
            score += 1.0 / log(degree)
    return score

def custom_resource_allocation_index(G, node1, node2):
    """Calculate Resource Allocation index with proper error handling"""
    common_neighbors = list(nx.common_neighbors(G, node1, node2))
    if not common_neighbors:
        return 0
    
    score = 0
    for w in common_neighbors:
        degree = G.degree(w)
        if degree > 0:  # Avoid division by zero
            score += 1.0 / degree
    return score

def custom_jaccard_coefficient(G, node1, node2):
    """Calculate Jaccard coefficient with proper error handling"""
    neighbors1 = set(G.neighbors(node1))
    neighbors2 = set(G.neighbors(node2))
    
    # Handle edge cases
    union_size = len(neighbors1.union(neighbors2))
    if union_size == 0:
        return 0
    
    return len(neighbors1.intersection(neighbors2)) / union_size

def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors with error handling"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 * norm2 > 0:
        return dot_product / (norm1 * norm2)
    else:
        return 0

In [31]:
def extract_features(G, node1, node2):
    """Extract features for a node pair using NetworkX graph with robust error handling"""
    features = []
    
    # Check if both nodes exist in the graph
    if node1 not in G or node2 not in G:
        # Return default features if nodes don't exist
        return [0] * 10  # Adjusted for feature count
    
    # Common neighbors
    common_neighbors = list(nx.common_neighbors(G, node1, node2))
    features.append(len(common_neighbors))
    
    # Jaccard coefficient
    features.append(custom_jaccard_coefficient(G, node1, node2))
    
    # Preferential attachment
    features.append(G.degree(node1) * G.degree(node2))
    
    # Adamic-Adar index
    features.append(custom_adamic_adar_index(G, node1, node2))
    
    # Resource allocation index
    features.append(custom_resource_allocation_index(G, node1, node2))
    
    # Shortest path length
    try:
        if G.has_edge(node1, node2):
            G.remove_edge(node1, node2)
            path_length = nx.shortest_path_length(G, node1, node2)
            G.add_edge(node1, node2)  # Restore the edge
        else:
            path_length = nx.shortest_path_length(G, node1, node2)
        features.append(path_length)
    except (nx.NetworkXNoPath, nx.NetworkXError):
        features.append(-1)  # No path exists
    
    # Node feature similarity
    if node1 in node_features and node2 in node_features:
        # Cosine similarity between node feature vectors
        cos_sim = cosine_similarity(node_features[node1], node_features[node2])
        features.append(cos_sim)
        
        # Correlation between node feature vectors
        try:
            corr = np.corrcoef(node_features[node1], node_features[node2])[0, 1]
            features.append(corr if not np.isnan(corr) else 0)
        except:
            features.append(0)
        
        # Number of matching non-zero features (indicating shared topics)
        n1_nonzero = node_features[node1] != 0
        n2_nonzero = node_features[node2] != 0
        matching_topics = np.logical_and(n1_nonzero, n2_nonzero).sum()
        features.append(matching_topics)
    else:
        # Default values if node features not available
        features.extend([0, 0, 0])
    
    # Community detection feature (shared neighbors ratio)
    try:
        neighbors1 = set(G.neighbors(node1))
        neighbors2 = set(G.neighbors(node2))
        if len(neighbors1) + len(neighbors2) > 0:
            shared_neighbors_ratio = len(neighbors1.intersection(neighbors2)) / (len(neighbors1) + len(neighbors2) - len(neighbors1.intersection(neighbors2)))
        else:
            shared_neighbors_ratio = 0
        features.append(shared_neighbors_ratio)
    except:
        features.append(0)
    
    return features

# IV. Prepare training data

In [32]:
print("Extracting features for training data...")
X = []
y = []

# Balance the dataset to avoid bias
min_examples = min(len(edges_positive), len(edges_negative))
print(f"Using {min_examples} examples from each class for balanced training")

# Positive examples
for src, dst in tqdm(edges_positive[:min_examples], desc="Processing positive examples"):
    features = extract_features(G, src, dst)
    X.append(features)
    y.append(1)

# Negative examples
for src, dst in tqdm(edges_negative[:min_examples], desc="Processing negative examples"):
    features = extract_features(G, src, dst)
    X.append(features)
    y.append(0)

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Extracting features for training data...
Using 5248 examples from each class for balanced training


Processing negative examples: 100%|██████████| 5248/5248 [00:00<00:00, 9217.74it/s]


# V. Train & performance training

In [33]:
print("Training model...")
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
with tqdm(total=1, desc="Training RandomForest") as pbar:
    model.fit(X_train, y_train)
    pbar.update(1)

Training model...


Training RandomForest: 100%|██████████| 1/1 [00:00<00:00,  4.17it/s]


In [ ]:
y_val_pred = model.predict(X_val)
print("\n----- Validation Results -----")
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Validation F1 Score: {f1_score(y_val, y_val_pred):.4f}")
print(f"Validation Precision: {precision_score(y_val, y_val_pred):.4f}")
print(f"Validation Recall: {recall_score(y_val, y_val_pred):.4f}")
print("Validation Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))


----- Validation Results -----
Validation Accuracy: 0.9210
Validation F1 Score: 0.9251
Validation Precision: 0.8791
Validation Recall: 0.9762
Validation Confusion Matrix:
[[ 909  141]
 [  25 1025]]


In [35]:
# Feature importance
feature_names = [
    "Common Neighbors", 
    "Jaccard Coefficient", 
    "Preferential Attachment", 
    "Adamic-Adar Index",
    "Resource Allocation Index", 
    "Shortest Path Length", 
    "Node Feature Cosine Similarity",
    "Node Feature Correlation",
    "Shared Topics Count",
    "Shared Neighbors Ratio"
]

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
print("\n----- Feature Importance -----")
for i in range(len(feature_names)):
    print(f"{feature_names[indices[i]]}: {importances[indices[i]]:.4f}")


----- Feature Importance -----
Preferential Attachment: 0.7889
Shortest Path Length: 0.1360
Jaccard Coefficient: 0.0184
Node Feature Correlation: 0.0181
Shared Neighbors Ratio: 0.0098
Node Feature Cosine Similarity: 0.0086
Adamic-Adar Index: 0.0081
Resource Allocation Index: 0.0068
Shared Topics Count: 0.0032
Common Neighbors: 0.0021


# Prediction on test data

In [36]:
print("\nGenerating predictions for test data...")
test_features = []
for src, dst in tqdm(test_pairs, desc="Processing test pairs"):
    test_features.append(extract_features(G, src, dst))


Generating predictions for test data...


Processing test pairs: 100%|██████████| 3498/3498 [00:00<00:00, 7637.97it/s]


In [37]:
print("Running prediction on test data...")
with tqdm(total=1, desc="Predicting") as pbar:
    test_predictions = model.predict(test_features)
    pbar.update(1)

Running prediction on test data...


Predicting: 100%|██████████| 1/1 [00:00<00:00, 17.91it/s]


In [38]:
print("\n----- Test Results -----")
print(f"Total test samples: {len(test_predictions)}")
print(f"Predicted positives (1): {np.sum(test_predictions == 1)}")
print(f"Predicted negatives (0): {np.sum(test_predictions == 0)}")
print(f"Positive rate: {np.mean(test_predictions):.4f}")

# Compare with random baseline
random_baseline_prob = 0.5
print(f"\nRandom baseline positive rate would be: {random_baseline_prob:.4f}")
print(f"Our model differs from random by: {abs(np.mean(test_predictions) - random_baseline_prob):.4f}")

with open("predictions.csv", "w") as pred_file:
    csv_out = csv.writer(pred_file)
    csv_out.writerow(["ID", "Predicted"])
    for i, prediction in enumerate(test_predictions):
        csv_out.writerow([i, int(prediction)])

print("\nPredictions saved to predictions.csv")

end_time = time.time()
print(f"\nTotal runtime: {end_time - start_time:.2f} seconds")

print("\n----- Summary -----")
print("Our feature-based model uses:")
print("1. Graph structure (common neighbors, path length, etc.)")
print("2. Node features from Wikipedia (932 text features per actor)")
print(f"Performance on validation set: {accuracy_score(y_val, y_val_pred):.4f} accuracy")
print(f"Expected improvement over random baseline: ~{(accuracy_score(y_val, y_val_pred) - 0.5):.4f}")


----- Test Results -----
Total test samples: 3498
Predicted positives (1): 1173
Predicted negatives (0): 2325
Positive rate: 0.3353

Random baseline positive rate would be: 0.5000
Our model differs from random by: 0.1647

Predictions saved to predictions.csv

Total runtime: 2.91 seconds

----- Summary -----
Our feature-based model uses:
1. Graph structure (common neighbors, path length, etc.)
2. Node features from Wikipedia (932 text features per actor)
Performance on validation set: 0.9210 accuracy
Expected improvement over random baseline: ~0.4210
